In [1]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data
spark = get_spark_session("getting_started")

yellow_taxi_data = get_yellow_taxi_data(spark=spark)

24/03/29 16:16:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# spark.sql("drop table if exists staging.yellow")
# spark.sql("drop schema if exists staging")


In [2]:
staging_sql = "CREATE DATABASE staging"
spark.sql(staging_sql)

DataFrame[]

In [3]:
yellow_taxi_data.write.saveAsTable("staging.yellow_taxi")

In [4]:
# if table exists we'll get an error
yellow_taxi_data.write.saveAsTable("staging.yellow_taxi")

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `staging`.`yellow_taxi` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

Let's check what we can see in [MinIO console](http://localhost:9001/browser)

In [5]:
!pip install minio


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
from minio import Minio
client = Minio(
        "minio:9000",
        access_key="admin",
        secret_key="password",
        secure=False  # Set to True if your Minio server uses TLS/SSL
    )

buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

warehouse 2024-03-29 15:48:16.681000+00:00


In [7]:
def print_objects_in_warehouse(bucket_name):
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
        print(obj.object_name, str(obj._size//1024) + "KB")
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 37251KB
staging/yellow_taxi/data/00001-2-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 44630KB
staging/yellow_taxi/metadata/00000-ae874524-b096-412f-8cb1-0c4760d6b2e9.metadata.json 3KB
staging/yellow_taxi/metadata/ab4678a7-c205-4a14-ac42-eceebd074368-m0.avro 8KB
staging/yellow_taxi/metadata/snap-4596078438615030908-1-ab4678a7-c205-4a14-ac42-eceebd074368.avro 4KB


## Note
As in Spark we can actually specify the mode: Append or Overwrite in order to avoid/mitigate or actually create some sort of solution for our target table

In [8]:
yellow_taxi_data.write.mode("overwrite").saveAsTable("staging.yellow_taxi")

## Note 2
But since we're using Iceberg we're getting a bit more!


In [9]:
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 37251KB
staging/yellow_taxi/data/00000-5-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 37251KB
staging/yellow_taxi/data/00001-2-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 44630KB
staging/yellow_taxi/data/00001-6-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 44630KB
staging/yellow_taxi/metadata/00000-ae874524-b096-412f-8cb1-0c4760d6b2e9.metadata.json 3KB
staging/yellow_taxi/metadata/00001-eeaf7be4-5a21-4c31-92cb-58020f16f80e.metadata.json 4KB
staging/yellow_taxi/metadata/ab4678a7-c205-4a14-ac42-eceebd074368-m0.avro 8KB
staging/yellow_taxi/metadata/db751d43-5f5e-4353-b847-bbd9bd57b54c-m0.avro 8KB
staging/yellow_taxi/metadata/snap-4596078438615030908-1-ab4678a7-c205-4a14-ac42-eceebd074368.avro 4KB
staging/yellow_taxi/metadata/snap-5363513689733971433-1-db751d43-5f5e-4353-b847-bbd9bd57b54c.avro 4KB


# Querying Data 

In [10]:
spark.sql("select * from staging.yellow_taxi").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-01-01 00:35:40|  2022-01-01 00:53:29|            2.0|          3.8|       1.0|                 N|         142|         236|           1|       14.5|  3.0|    0.5|      3.6

In [11]:
%%sql
select * from staging.yellow_taxi

24/03/29 16:17:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,N,68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0
1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.0,6.55,0.3,56.35,2.5,0.0
2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.2,0.0,0.3,26.0,2.5,0.0
2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.0,0.3,18.05,2.5,0.0
2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.0,0.0,0.3,8.8,2.5,0.0


In [12]:
%%sql
select count(1) as total_row_count from staging.yellow_taxi

total_row_count
5443362


# Schema Evolution
## *Note* Some of the content will be reused from Tabular.IO

In [13]:
%%sql
ALTER TABLE staging.yellow_taxi RENAME COLUMN fare_amount TO fare;


++
||
++
++

In [14]:
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 37251KB
staging/yellow_taxi/data/00000-5-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 37251KB
staging/yellow_taxi/data/00001-2-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 44630KB
staging/yellow_taxi/data/00001-6-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 44630KB
staging/yellow_taxi/metadata/00000-ae874524-b096-412f-8cb1-0c4760d6b2e9.metadata.json 3KB
staging/yellow_taxi/metadata/00001-eeaf7be4-5a21-4c31-92cb-58020f16f80e.metadata.json 4KB
staging/yellow_taxi/metadata/00002-0d55e1eb-df22-4269-a036-dde4cb852f3a.metadata.json 6KB
staging/yellow_taxi/metadata/ab4678a7-c205-4a14-ac42-eceebd074368-m0.avro 8KB
staging/yellow_taxi/metadata/db751d43-5f5e-4353-b847-bbd9bd57b54c-m0.avro 8KB
staging/yellow_taxi/metadata/snap-4596078438615030908-1-ab4678a7-c205-4a14-ac42-eceebd074368.avro 4KB
staging/yellow_taxi/metadata/snap-5363513689733971433-1-db751d43-5f5e-4353-b847-bbd9bd57b54c.avr

In [15]:
%%sql
select * from staging.yellow_taxi

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,N,68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0
1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.0,6.55,0.3,56.35,2.5,0.0
2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.2,0.0,0.3,26.0,2.5,0.0
2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.0,0.3,18.05,2.5,0.0
2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.0,0.0,0.3,8.8,2.5,0.0


In [16]:
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 37251KB
staging/yellow_taxi/data/00000-5-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 37251KB
staging/yellow_taxi/data/00001-2-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 44630KB
staging/yellow_taxi/data/00001-6-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 44630KB
staging/yellow_taxi/metadata/00000-ae874524-b096-412f-8cb1-0c4760d6b2e9.metadata.json 3KB
staging/yellow_taxi/metadata/00001-eeaf7be4-5a21-4c31-92cb-58020f16f80e.metadata.json 4KB
staging/yellow_taxi/metadata/00002-0d55e1eb-df22-4269-a036-dde4cb852f3a.metadata.json 6KB
staging/yellow_taxi/metadata/ab4678a7-c205-4a14-ac42-eceebd074368-m0.avro 8KB
staging/yellow_taxi/metadata/db751d43-5f5e-4353-b847-bbd9bd57b54c-m0.avro 8KB
staging/yellow_taxi/metadata/snap-4596078438615030908-1-ab4678a7-c205-4a14-ac42-eceebd074368.avro 4KB
staging/yellow_taxi/metadata/snap-5363513689733971433-1-db751d43-5f5e-4353-b847-bbd9bd57b54c.avr

In [17]:
%%sql
DELETE FROM staging.yellow_taxi
WHERE fare <1

++
||
++
++

## Let's see how our storage looks like!

In [18]:
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 37251KB
staging/yellow_taxi/data/00000-19-32e77607-82db-49c4-be96-67f47ec17724-0-00001.parquet 36874KB
staging/yellow_taxi/data/00000-5-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 37251KB
staging/yellow_taxi/data/00001-2-afaa0077-0731-4fd4-aed0-619421755d83-0-00001.parquet 44630KB
staging/yellow_taxi/data/00001-20-32e77607-82db-49c4-be96-67f47ec17724-0-00001.parquet 44147KB
staging/yellow_taxi/data/00001-6-3664f8f3-589d-44da-955f-cb1749f3a64a-0-00001.parquet 44630KB
staging/yellow_taxi/metadata/00000-ae874524-b096-412f-8cb1-0c4760d6b2e9.metadata.json 3KB
staging/yellow_taxi/metadata/00001-eeaf7be4-5a21-4c31-92cb-58020f16f80e.metadata.json 4KB
staging/yellow_taxi/metadata/00002-0d55e1eb-df22-4269-a036-dde4cb852f3a.metadata.json 6KB
staging/yellow_taxi/metadata/00003-b1914d19-e499-4aee-b269-01e0ddb3ee06.metadata.json 7KB
staging/yellow_taxi/metadata/85d522bf-4df1-48bb-bd49-bf17e4763200-m0.avro 

In [19]:
%%sql
select * from staging.yellow_taxi where fare <1

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee


## Why it's important and useful?

In [20]:
%%sql
select * from staging.yellow_taxi.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2024-03-29 16:16:54.154000,4596078438615030908,None,append,s3://warehouse/staging/yellow_taxi/metadata/snap-4596078438615030908-1-ab4678a7-c205-4a14-ac42-eceebd074368.avro,"{'spark.app.id': 'local-1711728998183', 'changed-partition-count': '1', 'added-data-files': '2', 'total-equality-deletes': '0', 'added-records': '5443362', 'total-position-deletes': '0', 'added-files-size': '83846720', 'total-delete-files': '0', 'total-files-size': '83846720', 'total-records': '5443362', 'total-data-files': '2'}"
2024-03-29 16:17:14.982000,5363513689733971433,None,append,s3://warehouse/staging/yellow_taxi/metadata/snap-5363513689733971433-1-db751d43-5f5e-4353-b847-bbd9bd57b54c.avro,"{'spark.app.id': 'local-1711728998183', 'changed-partition-count': '1', 'added-data-files': '2', 'total-equality-deletes': '0', 'added-records': '5443362', 'total-position-deletes': '0', 'added-files-size': '83846720', 'total-delete-files': '0', 'total-files-size': '83846720', 'total-records': '5443362', 'total-data-files': '2'}"
2024-03-29 16:17:54.145000,9160540137384676132,5363513689733971433,overwrite,s3://warehouse/staging/yellow_taxi/metadata/snap-9160540137384676132-1-85d522bf-4df1-48bb-bd49-bf17e4763200.avro,"{'added-data-files': '2', 'total-equality-deletes': '0', 'added-records': '5412306', 'deleted-data-files': '2', 'deleted-records': '5443362', 'total-records': '5412306', 'spark.app.id': 'local-1711728998183', 'removed-files-size': '83846720', 'changed-partition-count': '1', 'total-position-deletes': '0', 'added-files-size': '82967139', 'total-delete-files': '0', 'total-files-size': '82967139', 'total-data-files': '2'}"


## Let's query the data where we still had very first version

In [ ]:
%%sql
select * from staging.yellow_taxi VERSION AS OF 7958142262214527681;

## Now Image we need to rollback to a certain version!

Let's check how bucket information looks like first!

In [ ]:
print_objects_in_warehouse("warehouse")

BTW: fun trick with cell magic!

In [ ]:
%%sql --var df
select * from staging.yellow_taxi.history

In [ ]:
df.head().snapshot_id

In [ ]:

spark.sql("""CALL system.rollback_to_snapshot('staging.yellow_taxi', 2204675250161940923)""")

In [ ]:
%%sql
select * from staging.yellow_taxi where fare < 1

In [ ]:
print_objects_in_warehouse("warehouse")